In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import pytz
from IPython.display import display

BASE_URL = "https://twelvedata.com/markets/300755/commodity/xau-usd/historical-data"
START_DATE = "2024-10-01"
END_DATE = "2024-10-31"
INTERVAL = "15min"

def fetch_html(url):
    response = requests.get(url)
    if response.status_code == 200:
        print(f"Page fetched successfully: {url}")
        return response.text
    else:
        print(f"Failed to fetch page: {response.status_code}")
        return None

def parse_html(html):
    soup = BeautifulSoup(html, "html.parser")
    table = soup.find("table")
    if not table:
        print("No table found on the page.")
        return None, None

    headers = [header.text.strip() for header in table.find_all("th")]
    rows = [[col.text.strip() for col in row.find_all("td")] for row in table.find_all("tr")[1:]]
    return headers, rows

def convert_to_utc_ms(datetime_str, timezone="US/Eastern"):
    naive_datetime = datetime.strptime(datetime_str, "%b %d, %Y %H:%M")
    local_timezone = pytz.timezone(timezone)
    localized_datetime = local_timezone.localize(naive_datetime)
    utc_datetime = localized_datetime.astimezone(pytz.utc)
    return int(utc_datetime.timestamp() * 1000)

def convert_to_decimal(value):
    """Converts K values to full decimal numbers."""
    if "K" in value:
        return round(float(value.replace("K", "")) * 1000, 1)
    else:
        return round(float(value), 1)

def process_all_pages(base_url, start_date, end_date, interval, total_pages):
    all_data = []
    for page_num in range(1, total_pages + 1):
        print(f"\nProcessing Page {page_num}...")
        url = f"{base_url}?start_date={start_date}&end_date={end_date}&interval={interval}&page={page_num}"
        html_content = fetch_html(url)
        if not html_content:
            print(f"Skipping page {page_num} due to an error.")
            continue

        headers, rows = parse_html(html_content)
        if not headers or not rows:
            print(f"No data found on page {page_num}.")
            continue

        raw_df = pd.DataFrame(rows, columns=headers)
        raw_df.rename(columns={
            "Date": "DateTime",
            "Open": "Candle Start Price",
            "High": "Candle Max Price",
            "Low": "Candle Min Price",
            "Close": "Candle Close Price"
        }, inplace=True)

        # Convert values to decimals and DateTime to ms
        raw_df["Candle Start Price"] = raw_df["Candle Start Price"].apply(convert_to_decimal)
        raw_df["Candle Max Price"] = raw_df["Candle Max Price"].apply(convert_to_decimal)
        raw_df["Candle Min Price"] = raw_df["Candle Min Price"].apply(convert_to_decimal)
        raw_df["Candle Close Price"] = raw_df["Candle Close Price"].apply(convert_to_decimal)
        raw_df["Time in MS"] = raw_df["DateTime"].apply(lambda x: convert_to_utc_ms(x))

        processed_df = raw_df[["Time in MS", "Candle Start Price", "Candle Close Price", "Candle Max Price", "Candle Min Price"]]

        all_data.append(processed_df)

        # Print first and last 5 rows for this page
        print(f"\nPage {page_num} Raw Data (First 5 Rows):\n")
        print(raw_df.head())
        print(f"\nPage {page_num} Raw Data (Last 5 Rows):\n")
        print(raw_df.tail())

        print(f"\nPage {page_num} Processed Data (First 5 Rows):\n")
        print(processed_df.head())
        print(f"\nPage {page_num} Processed Data (Last 5 Rows):\n")
        print(processed_df.tail())

    print("\nCombining all pages...")
    combined_df = pd.concat(all_data, ignore_index=True)
    print("All pages combined successfully.")

    print("\nSorting the combined table by Time in MS in ascending order...")
    sorted_df = combined_df.sort_values(by="Time in MS", ascending=True).reset_index(drop=True)
    print("Final table sorted successfully.")

    print("\nFinal Sorted Table (First 5 Rows):\n")
    print(sorted_df.head())
    print("\nFinal Sorted Table (Last 5 Rows):\n")
    print(sorted_df.tail())

    return sorted_df

def beautify_table(df):
    """
    Beautifies the table with black headers, white text, center alignment, and bold text for values.
    Displays the first and last 5 rows of the DataFrame.
    """
    print("\nDisplaying First 5 Rows of Final Table (Formatted):\n")
    display(
        df.head(5).style.set_properties(**{
            "text-align": "center",
            "font-weight": "bold"
        }).set_table_styles([
            {"selector": "th", "props": [("background-color", "black"), ("color", "white"), ("text-align", "center")]},
            {"selector": "td", "props": [("border", "1px solid black")]}
        ])
    )

    print("\nDisplaying Last 5 Rows of Final Table (Formatted):\n")
    display(
        df.tail(5).style.set_properties(**{
            "text-align": "center",
            "font-weight": "bold"
        }).set_table_styles([
            {"selector": "th", "props": [("background-color", "black"), ("color", "white"), ("text-align", "center")]},
            {"selector": "td", "props": [("border", "1px solid black")]}
        ])
    )

def beautify_and_export_table(df, filename):
    print("Applying formatting to the final table...")
    beautify_table(df)  # Prints the beautified first and last 5 rows
    print("\nExporting final table to CSV...")
    df.to_csv(filename, index=False)
    print(f"Final sorted and formatted data exported to '{filename}'.")

def main():
    TOTAL_PAGES = 22
    final_table = process_all_pages(BASE_URL, START_DATE, END_DATE, INTERVAL, TOTAL_PAGES)
    beautify_and_export_table(final_table, "final_sorted_data.csv")

if __name__ == "__main__":
    main()



Processing Page 1...
Page fetched successfully: https://twelvedata.com/markets/300755/commodity/xau-usd/historical-data?start_date=2024-10-01&end_date=2024-10-31&interval=15min&page=1

Page 1 Raw Data (First 5 Rows):

             DateTime  Candle Start Price  Candle Max Price  Candle Min Price  \
0  Oct 31, 2024 23:45              2774.4            2775.1            2772.1   
1  Oct 31, 2024 23:30              2777.4            2778.9            2774.2   
2  Oct 31, 2024 23:15              2781.6            2782.0            2776.1   
3  Oct 31, 2024 23:00              2779.6            2781.7            2779.1   
4  Oct 31, 2024 22:45              2778.9            2780.3            2777.1   

   Candle Close Price  % Change     Time in MS  
0              2773.6  -0.0303%  1730432700000  
1              2774.4  -0.1109%  1730431800000  
2              2777.5  -0.1445%  1730430900000  
3              2781.6   0.0723%  1730430000000  
4              2779.6   0.0259%  1730429100000  


,Time in MS,Candle Start Price,Candle Close Price,Candle Max Price,Candle Min Price
0,1727755200000,2635.700000,2630.800000,2635.800000,2629.500000
1,1727756100000,2630.700000,2632.000000,2634.200000,2628.900000
2,1727757000000,2632.200000,2633.700000,2634.700000,2630.600000
3,1727757900000,2633.800000,2633.400000,2636.100000,2633.400000
4,1727758800000,2633.400000,2633.700000,2634.400000,2630.400000



Displaying Last 5 Rows of Final Table (Formatted):



,Time in MS,Candle Start Price,Candle Close Price,Candle Max Price,Candle Min Price
2107,1730429100000,2778.900000,2779.600000,2780.300000,2777.100000
2108,1730430000000,2779.600000,2781.600000,2781.700000,2779.100000
2109,1730430900000,2781.600000,2777.500000,2782.000000,2776.100000
2110,1730431800000,2777.400000,2774.400000,2778.900000,2774.200000
2111,1730432700000,2774.400000,2773.600000,2775.100000,2772.100000



Exporting final table to CSV...
Final sorted and formatted data exported to 'final_sorted_data.csv'.


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import pytz
from IPython.display import display

BASE_URL = "https://twelvedata.com/markets/300755/commodity/xau-usd/historical-data"
START_DATE = "2024-10-01"
END_DATE = "2024-10-31"
INTERVAL = "1h"  # Changed to 1-hour interval
TOTAL_PAGES = 6  # Adjusted for 6 pages

def fetch_html(url):
    response = requests.get(url)
    if response.status_code == 200:
        print(f"Page fetched successfully: {url}")
        return response.text
    else:
        print(f"Failed to fetch page: {response.status_code}")
        return None

def parse_html(html):
    soup = BeautifulSoup(html, "html.parser")
    table = soup.find("table")
    if not table:
        print("No table found on the page.")
        return None, None

    headers = [header.text.strip() for header in table.find_all("th")]
    rows = [[col.text.strip() for col in row.find_all("td")] for row in table.find_all("tr")[1:]]
    return headers, rows

def convert_to_utc_ms(datetime_str, timezone="US/Eastern"):
    naive_datetime = datetime.strptime(datetime_str, "%b %d, %Y %H:%M")
    local_timezone = pytz.timezone(timezone)
    localized_datetime = local_timezone.localize(naive_datetime)
    utc_datetime = localized_datetime.astimezone(pytz.utc)
    return int(utc_datetime.timestamp() * 1000)

def convert_to_decimal(value):
    """Converts K values to full decimal numbers."""
    if "K" in value:
        return round(float(value.replace("K", "")) * 1000, 1)
    else:
        return round(float(value), 1)

def process_all_pages(base_url, start_date, end_date, interval, total_pages):
    all_data = []
    for page_num in range(1, total_pages + 1):
        print(f"\nProcessing Page {page_num}...")
        url = f"{base_url}?start_date={start_date}&end_date={end_date}&interval={interval}&page={page_num}"
        html_content = fetch_html(url)
        if not html_content:
            print(f"Skipping page {page_num} due to an error.")
            continue

        headers, rows = parse_html(html_content)
        if not headers or not rows:
            print(f"No data found on page {page_num}.")
            continue

        raw_df = pd.DataFrame(rows, columns=headers)
        raw_df.rename(columns={
            "Date": "DateTime",
            "Open": "Candle Start Price",
            "High": "Candle Max Price",
            "Low": "Candle Min Price",
            "Close": "Candle Close Price"
        }, inplace=True)

        # Convert values to decimals and DateTime to ms
        raw_df["Candle Start Price"] = raw_df["Candle Start Price"].apply(convert_to_decimal)
        raw_df["Candle Max Price"] = raw_df["Candle Max Price"].apply(convert_to_decimal)
        raw_df["Candle Min Price"] = raw_df["Candle Min Price"].apply(convert_to_decimal)
        raw_df["Candle Close Price"] = raw_df["Candle Close Price"].apply(convert_to_decimal)
        raw_df["Time in MS"] = raw_df["DateTime"].apply(lambda x: convert_to_utc_ms(x))

        processed_df = raw_df[["Time in MS", "Candle Start Price", "Candle Close Price", "Candle Max Price", "Candle Min Price"]]

        all_data.append(processed_df)

        # Print first and last 5 rows for this page
        print(f"\nPage {page_num} Raw Data (First 5 Rows):\n")
        print(raw_df.head())
        print(f"\nPage {page_num} Raw Data (Last 5 Rows):\n")
        print(raw_df.tail())

        print(f"\nPage {page_num} Processed Data (First 5 Rows):\n")
        print(processed_df.head())
        print(f"\nPage {page_num} Processed Data (Last 5 Rows):\n")
        print(processed_df.tail())

    print("\nCombining all pages...")
    combined_df = pd.concat(all_data, ignore_index=True)
    print("All pages combined successfully.")

    print("\nSorting the combined table by Time in MS in ascending order...")
    sorted_df = combined_df.sort_values(by="Time in MS", ascending=True).reset_index(drop=True)
    print("Final table sorted successfully.")

    print("\nFinal Sorted Table (First 5 Rows):\n")
    print(sorted_df.head())
    print("\nFinal Sorted Table (Last 5 Rows):\n")
    print(sorted_df.tail())

    return sorted_df

def beautify_table(df):
    """
    Beautifies the table with black headers, white text, center alignment, and bold text for values.
    Displays the first and last 5 rows of the DataFrame.
    """
    print("\nDisplaying First 5 Rows of Final Table (Formatted):\n")
    display(
        df.head(5).style.set_properties(**{
            "text-align": "center",
            "font-weight": "bold"
        }).set_table_styles([
            {"selector": "th", "props": [("background-color", "black"), ("color", "white"), ("text-align", "center")]},
            {"selector": "td", "props": [("border", "1px solid black")]}
        ])
    )

    print("\nDisplaying Last 5 Rows of Final Table (Formatted):\n")
    display(
        df.tail(5).style.set_properties(**{
            "text-align": "center",
            "font-weight": "bold"
        }).set_table_styles([
            {"selector": "th", "props": [("background-color", "black"), ("color", "white"), ("text-align", "center")]},
            {"selector": "td", "props": [("border", "1px solid black")]}
        ])
    )

def beautify_and_export_table(df, filename):
    print("Applying formatting to the final table...")
    beautify_table(df)  # Prints the beautified first and last 5 rows
    print("\nExporting final table to CSV...")
    df.to_csv(filename, index=False)
    print(f"Final sorted and formatted data exported to '{filename}'.")

def main():
    final_table = process_all_pages(BASE_URL, START_DATE, END_DATE, INTERVAL, TOTAL_PAGES)
    beautify_and_export_table(final_table, "final_sorted_1h_data.csv")

if __name__ == "__main__":
    main()



Processing Page 1...
Page fetched successfully: https://twelvedata.com/markets/300755/commodity/xau-usd/historical-data?start_date=2024-10-01&end_date=2024-10-31&interval=1h&page=1

Page 1 Raw Data (First 5 Rows):

             DateTime  Candle Start Price  Candle Max Price  Candle Min Price  \
0  Oct 31, 2024 23:00              2779.6            2782.0            2772.1   
1  Oct 31, 2024 22:00              2781.0            2782.7            2776.8   
2  Oct 31, 2024 21:00              2778.3            2781.3            2773.4   
3  Oct 31, 2024 20:00              2778.5            2780.3            2777.2   
4  Oct 31, 2024 19:00              2782.2            2783.2            2777.1   

   Candle Close Price  % Change     Time in MS  
0              2773.6  -0.2159%  1730430000000  
1              2779.6  -0.0503%  1730426400000  
2              2781.0   0.0993%  1730422800000  
3              2778.3  -0.0076%  1730419200000  
4              2779.2  -0.1093%  1730415600000  

Pa

,Time in MS,Candle Start Price,Candle Close Price,Candle Max Price,Candle Min Price
0,1727755200000,2635.700000,2633.400000,2636.100000,2628.900000
1,1727758800000,2633.400000,2633.900000,2636.500000,2630.400000
2,1727762400000,2634.100000,2639.100000,2639.500000,2633.500000
3,1727766000000,2639.300000,2638.000000,2641.800000,2636.500000
4,1727769600000,2637.800000,2631.600000,2638.400000,2624.800000



Displaying Last 5 Rows of Final Table (Formatted):



,Time in MS,Candle Start Price,Candle Close Price,Candle Max Price,Candle Min Price
523,1730415600000,2782.200000,2779.200000,2783.200000,2777.100000
524,1730419200000,2778.500000,2778.300000,2780.300000,2777.200000
525,1730422800000,2778.300000,2781.000000,2781.300000,2773.400000
526,1730426400000,2781.000000,2779.600000,2782.700000,2776.800000
527,1730430000000,2779.600000,2773.600000,2782.000000,2772.100000



Exporting final table to CSV...
Final sorted and formatted data exported to 'final_sorted_1h_data.csv'.
